In [1]:
%load_ext autoreload
# to make modification in .py file type %autoreload

%load_ext line_profiler
#for finding out how long code takes to run
#an example of how to get diagnostics on functions
#%lprun -f ff.generate_map_full ff.generate_map_full(384, A_BB, alpha_BB, beta_sigma, freqs, nu0)

In [2]:
#trying to consolidate the work so far into a cohesive document and notation. Pushing definitions into 
#separate .py file

#------packages-------------------
%pylab inline
import time
import healpy as hp
import matplotlib.pyplot as plt
import foreground_functions as ff
from scipy.optimize import curve_fit, minimize

Populating the interactive namespace from numpy and matplotlib


In [5]:
nside = 128
A = 1.7e3
alpha = -3.0
beta = -3.2
gamma = -2.5
ells = np.arange(3*nside)
freqs = np.linspace(30, 300, 5)*1.e9
filepath = '/Users/alex/Documents/foreground_moments/power_spectra'

In [11]:
%autoreload

In [7]:
# a = ff.get_wigner_sum()
# save(filepath + 'default_wigner_sum.npy', a)

In [8]:
a = load(filepath + 'default_wigner_sum.npy')

In [10]:
print(a-ff.get_wigner_sum())

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.

In [22]:
cls = np.array([0,1,2,3,4,5,6,7,8,9])

In [19]:
cls_2d = cls[newaxis, newaxis]

In [20]:
cls_2d.shape

(1, 1, 10)

In [21]:
cls_2d

array([[[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]]])

In [23]:
cls[newaxis] = cls

In [24]:
cls

array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

In [108]:
#for fun let's imagine 
#ell runs from 0 to 10
#ell1 runs fomr 0 to 5
#ell2 runs from 0 to 5

In [132]:
w3j = np.ones((10,10,10))

In [159]:
#the cls we want to insert along the ell1 direction, so the should have length 5
amp_cls = np.arange(10, 20)
#beta_cls go along the ell2 direction so have length 7
beta_cls = np.arange(10)

(10,)


In [113]:
#test what multiplications are allowed

In [114]:
b = w3j[0,:,0]*amp_cls
c = w3j[:,:,0]*amp_cls
d = amp_cls*w3j[:,:,0]
print(b.shape, c.shape, d.shape)

(5,) (10, 5) (10, 5)


In [115]:
%autoreload

In [116]:
start=time.time()
ff.get_wigner_sum()
print(time.time()-start)

(384, 384, 384)
(384,)
(384,)
(384, 384, 384)
(512, 384, 512)
(512, 384, 512)
3.3283560276031494


In [162]:
big_amp = np.ones_like(w3j)

In [191]:
start=time.time()
big_amp = np.zeros((384,384,384))
big_beta = big_amp
amp_cls_2 = np.arange(384)
beta_cls_2 = np.arange(384)
N=384
for i in range(N):
    for j in range(N):
        big_amp[i,:,j] = amp_cls_2
        big_beta[i, j, :] = beta_cls_2
c = big_beta * big_amp
d = sum(c, (1,2))
#     big_amp[i,:,:] *= amp_cls
#     big_amp[:,:,i] *= amp_cls
# big_amp *= 0.1
# big_amp *= 0.5
# big_amp = np.sqrt(big_amp)
print(time.time()-start)

1.015523910522461


In [184]:
for i in range(10):
    print(big_amp[0,:,i])

[10. 11. 12. 13. 14. 15. 16. 17. 18. 19.]
[10. 11. 12. 13. 14. 15. 16. 17. 18. 19.]
[10. 11. 12. 13. 14. 15. 16. 17. 18. 19.]
[10. 11. 12. 13. 14. 15. 16. 17. 18. 19.]
[10. 11. 12. 13. 14. 15. 16. 17. 18. 19.]
[10. 11. 12. 13. 14. 15. 16. 17. 18. 19.]
[10. 11. 12. 13. 14. 15. 16. 17. 18. 19.]
[10. 11. 12. 13. 14. 15. 16. 17. 18. 19.]
[10. 11. 12. 13. 14. 15. 16. 17. 18. 19.]
[10. 11. 12. 13. 14. 15. 16. 17. 18. 19.]


In [127]:
big_amp[:,0,:]

array([[0., 1., 2., 3., 4.],
       [0., 1., 2., 3., 4.],
       [0., 1., 2., 3., 4.],
       [0., 1., 2., 3., 4.],
       [0., 1., 2., 3., 4.],
       [0., 1., 2., 3., 4.],
       [0., 1., 2., 3., 4.],
       [0., 1., 2., 3., 4.],
       [0., 1., 2., 3., 4.],
       [0., 1., 2., 3., 4.]])